# Age discrepancy

Now we will try to correlate age discrepancy with some scores of the behavioral dataset. Our model to predict age uses only the structural MRI features.

In [1]:
import warnings
warnings.filterwarnings('ignore')
from utils import create_dataset_mri, cv_for_cde, create_dataset_eeg
from cde.density_estimator import MixtureDensityNetwork
import numpy as np
import tensorflow as tf
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tensorflow.python.keras.activations import tanh
from sklearn.impute import SimpleImputer
import pandas as pd
import tensorflow as tf
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Fix age threshold
threshold = 16

In [3]:
# Behavioral data
behavioral = pd.read_csv('data/Behavioral/cleaned/HBNFinalSummaries.csv')
# Create dataset MRI
target = 'Age'
data = create_dataset_mri(SCORE = target)

test = data.loc[data['DX_01'].isin(['Autism Spectrum Disorder', 'ADHD-Combined Type', 'ADHD-Inattentive Type', 'Specific Learning Disorder with Impairment in Reading'])]
healthy = data.loc[data['DX_01'].isin(['No Diagnosis Given'])]
train = data.loc[~data['DX_01'].isin(['Autism Spectrum Disorder', 'ADHD-Combined Type', 'ADHD-Inattentive Type', 'No Diagnosis Given', 'Specific Learning Disorder with Impairment in Reading'])]

test = test = test[test['Age']<threshold]
healthy = healthy[healthy['Age']<threshold]
train = train[train['Age']<threshold]


train.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
healthy.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
test.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)



In [4]:
# train
train = np.array(train)
ID_train_init = train[:,0]
X_train = train[:,2:]
y_train = train[:, 1]
y_train = y_train.reshape((-1,1))

# test
test = np.array(test)
ID_test_init = test[:,0]
X_test = test[:,2:]
y_test = test[:, 1]
y_test = y_test.reshape((-1,1))

# healthy
healthy = np.array(healthy)
y_healthy = healthy[:, 1]
X_healthy = np.concatenate((np.reshape(healthy[:,0],[-1,1]), healthy[:,2:]), axis = 1)
y_healthy = y_healthy.reshape((-1,1))


X_test_init = np.array(X_test, dtype=np.float64)
y_test_init = np.array(y_test, dtype=np.float64)
X_train_init = np.array(X_train, dtype=np.float64)
y_train_init  = np.array(y_train, dtype=np.float64)

# split the healthy
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_healthy, y_healthy, test_size=0.5, random_state=8)
y_train_h = y_train_h.reshape((-1,1))
y_test_h = y_test_h.reshape((-1,1))
ID_train_h = X_train_h[:,0]
X_train_h = X_train_h[:,1:]
ID_test_h = X_test_h[:,0]
X_test_h = X_test_h[:,1:]
y_train_h = np.array(y_train_h, dtype=np.float64)
X_train_h = np.array(X_train_h, dtype=np.float64)
y_test_h = np.array(y_test_h, dtype=np.float64)
X_test_h = np.array(X_test_h, dtype=np.float64)
# Now add again
ID_test = np.concatenate((ID_test_init, ID_test_h))
y_test = np.concatenate((y_test_init, y_test_h))
X_test = np.concatenate((X_test_init, X_test_h))

ID_train = np.concatenate((ID_train_init, ID_train_h))
y_train = np.concatenate((y_train_init, y_train_h))
X_train = np.concatenate((X_train_init, X_train_h))

In [5]:
# Set model parameters
ndim_x=X_train.shape[1]
ndim_y=y_train.shape[1]
# We try the "faster decay rate for non-gaussian data" proposed in the paper: h = n^(-1/(d+1))
n = X_train.shape[0]
d = X_train.shape[1]+y_train.shape[1]
h = n**(-1/(d+1))
# Define the model
model = MixtureDensityNetwork('MDN', ndim_x, ndim_y, n_centers=10, hidden_sizes=(16, 16), hidden_nonlinearity=tf.nn.tanh,
               n_training_epochs=1000, x_noise_std=h, y_noise_std=h, adaptive_noise_fn=None, entropy_reg_coef=0.0,
               weight_decay=0.0, weight_normalization=True, data_normalization=True, dropout=0.0, l2_reg=0.0, l1_reg=0.0,
               random_seed=42)
# Fit
model.fit(X_train, y_train)
# Predict
y_pred = model.mean_(X_test)
y_pred = y_pred.reshape((-1,1))
y_pred.shape
print('Test MSE: {}'.format(mean_squared_error(y_pred, y_test)))

1000/1000 [100%] ██████████████████████████████ Elapsed: 18s | loss: 653.036
mean log-loss train: 1.4577
Test MSE: 3.653655608823112


In [6]:
# Define discrepancy
std = model.std_(X_test)
discrepancy = np.divide((y_test-y_pred), 1+std)
#discrepancy = y_test-y_pred

In [7]:
# Get dataframe for test observations with behavioral data + discrepancy
data = {'discrepancy':discrepancy[:,0]}
discrepancy_df = pd.DataFrame(data)
ID_df = pd.DataFrame({'EID':ID_test})
discrepancy_merged = pd.concat([ID_df, discrepancy_df], axis=1)
dataframe = pd.merge(discrepancy_merged, behavioral, how='inner', on='EID')
dataframe = dataframe.drop(['Anonymized.ID', 'Study.Site'], axis = 1)

In [59]:
# Get the correlations
correlations = dataframe[dataframe.columns[1:]].corr()['discrepancy'][:]
# Inspect correlations
correlations[correlations > 0.25]  

discrepancy              1.000000
Age                      0.451420
DX_03_New                0.259807
DX_04_New                0.340653
DX_04_Rem                0.323752
CBCL_Pre_WD              0.253926
CELF_SA_R                0.277746
CLEF5M_FL_Raw            0.273480
Picture_Seq_Raw          0.261719
PPVT4_RawScore           0.296596
sib3dxse                 0.484766
TRF_Pre_Attention_Raw    0.359634
VL_Comm1_Raw             0.574106
VL_Comm_Stnd             0.808613
VL_DLS_Scale             0.298061
VL_Social_Scale          0.554523
WIAT_Num_Raw             0.311534
WIAT_Spell_Raw           0.283371
WIAT_MP_Raw              0.289897
Name: discrepancy, dtype: float64

In [55]:
dataframe.shape

(566, 446)

In [72]:
relevant_corr = []
for feat in correlations[correlations > 0.20].index:
    if dataframe[feat].isna().sum() < 100:
        relevant_corr.append(feat)
relevant_corr

['discrepancy',
 'Age',
 'CTOPP_EL_R',
 'CTOPP_BW_R',
 'FGC_Curl_Up',
 'PCIAT_Total',
 'WIAT_Num_Raw',
 'WIAT_Spell_Raw',
 'WIAT_Word_Raw',
 'WIAT_LCRV_Raw',
 'WIAT_MP_Raw',
 'WISC_BD_Raw']

It looks like that there is an interesting correlation between Wechsler Individual Achievement Test (WIAT) and the age discrepancy. Let us now study whether this relationship is not due to the correlation between age and age discrepancy. I.e. let us see if the age discrepancy is giving us non-redundant (w.r.t the age) information about WIAT.

In [73]:
age_correlations = dataframe[dataframe.columns[1:]].corr()['Age'][:]
age_correlations[relevant_corr]

discrepancy       0.451420
Age               1.000000
CTOPP_EL_R        0.430704
CTOPP_BW_R        0.441091
FGC_Curl_Up       0.577915
PCIAT_Total       0.425690
WIAT_Num_Raw      0.778509
WIAT_Spell_Raw    0.713854
WIAT_Word_Raw     0.639737
WIAT_LCRV_Raw     0.618837
WIAT_MP_Raw       0.731364
WISC_BD_Raw       0.530942
Name: Age, dtype: float64

There is high correlation between age and WIAT, but for CTOPP and PCIAT it is lower. 

In [92]:
model = ols('WIAT_Word_Raw ~ discrepancy + Age', data = dataframe).fit()
                
anova_result = sm.stats.anova_lm(model, typ=2)
print (anova_result)

                    sum_sq     df           F        PR(>F)
discrepancy     597.893626    1.0    2.874048  9.061228e-02
Age           68750.404531    1.0  330.480090  1.276242e-57
Residual     109008.721299  524.0         NaN           NaN


In [91]:
pvals = dict()

for feat in relevant_corr:
    model = ols('{} ~ discrepancy + Age'.format(feat), data = dataframe).fit()
    anova_result = sm.stats.anova_lm(model, typ=2)
    
    pvals[feat] = anova_result['PR(>F)']['discrepancy']
                
pvals

{'discrepancy': 0.0,
 'Age': 1.0,
 'CTOPP_EL_R': 0.7605612970528647,
 'CTOPP_BW_R': 0.8978141472047116,
 'FGC_Curl_Up': 0.4141129979267989,
 'PCIAT_Total': 0.3882321985743732,
 'WIAT_Num_Raw': 0.09744599509459974,
 'WIAT_Spell_Raw': 0.15299767574839895,
 'WIAT_Word_Raw': 0.09061227711565088,
 'WIAT_LCRV_Raw': 0.05194419477173275,
 'WIAT_MP_Raw': 0.06642134014895226,
 'WISC_BD_Raw': 0.861677651437721}

## Consider the most frequent situations: ADHD Inattentive, ADHD Combined, Autism, Healthy. Specific impairment in Reading

In [13]:
df = dataframe.loc[dataframe['DX_01'].isin(['ADHD-Combined Type', 'ADHD-Inattentive Type','No Diagnosis Given', 'Autism Spectrum Disorder', 'Specific Learning Disorder with Impairment in Reading'])]
df = df[df['Age']<threshold]

## Evaluate these quantities as an average over different splits

In [154]:
def discrepancy_corr(repetitions, multiple_testing_correction = 'fdr_bh', exclude_old = True, threshold = threshold, corr_threshold = 0.2):
    # Behavioral data
    behavioral = pd.read_csv('data/Behavioral/cleaned/HBNFinalSummaries.csv')
    # Create dataset MRI
    target = 'Age'
    data = create_dataset_mri(SCORE = target)

    test = data.loc[data['DX_01'].isin(['Autism Spectrum Disorder', 'ADHD-Combined Type', 'ADHD-Inattentive Type', 'Specific Learning Disorder with Impairment in Reading'])]
    healthy = data.loc[data['DX_01'].isin(['No Diagnosis Given'])]
    train = data.loc[~data['DX_01'].isin(['Autism Spectrum Disorder', 'ADHD-Combined Type', 'ADHD-Inattentive Type', 'No Diagnosis Given', 'Specific Learning Disorder with Impairment in Reading'])]
    
    if exclude_old == True:
        # Remove patients aged > threshold
        test = test = test[test['Age']<threshold]
        healthy = healthy[healthy['Age']<threshold]
        train = train[train['Age']<threshold]
    
    
    train.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
    healthy.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
    test.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)

    
    # train
    train = np.array(train)
    ID_train_init = train[:,0]
    X_train = train[:,2:]
    y_train = train[:, 1]
    y_train = y_train.reshape((-1,1))

    # test
    test = np.array(test)
    ID_test_init = test[:,0]
    X_test = test[:,2:]
    y_test = test[:, 1]
    y_test = y_test.reshape((-1,1))

    # healthy
    healthy = np.array(healthy)
    y_healthy = healthy[:, 1]
    X_healthy = np.concatenate((np.reshape(healthy[:,0],[-1,1]), healthy[:,2:]), axis = 1)
    y_healthy = y_healthy.reshape((-1,1))

    
    X_test_init = np.array(X_test, dtype=np.float64)
    y_test_init = np.array(y_test, dtype=np.float64)
    X_train_init = np.array(X_train, dtype=np.float64)
    y_train_init  = np.array(y_train, dtype=np.float64)
    
    # things I want to compute
    t_adhd = []
    t_adhd_combined = []
    t_autism = [] 
    t_impaired = []
    
    pvals = dict()
    
    for i in range(repetitions):
        # split the healthy
        X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_healthy, y_healthy, test_size=0.5, random_state=i)
        y_train_h = y_train_h.reshape((-1,1))
        y_test_h = y_test_h.reshape((-1,1))
        ID_train_h = X_train_h[:,0]
        X_train_h = X_train_h[:,1:]
        ID_test_h = X_test_h[:,0]
        X_test_h = X_test_h[:,1:]
        y_train_h = np.array(y_train_h, dtype=np.float64)
        X_train_h = np.array(X_train_h, dtype=np.float64)
        y_test_h = np.array(y_test_h, dtype=np.float64)
        X_test_h = np.array(X_test_h, dtype=np.float64)
        # Now add again
        ID_test = np.concatenate((ID_test_init, ID_test_h))
        y_test = np.concatenate((y_test_init, y_test_h))
        X_test = np.concatenate((X_test_init, X_test_h))

        ID_train = np.concatenate((ID_train_init, ID_train_h))
        y_train = np.concatenate((y_train_init, y_train_h))
        X_train = np.concatenate((X_train_init, X_train_h))
    
        # Set model parameters
        ndim_x=X_train.shape[1]
        ndim_y=y_train.shape[1]
        # We try the "faster decay rate for non-gaussian data" proposed in the paper: h = n^(-1/(d+1))
        n = X_train.shape[0]
        d = X_train.shape[1]+y_train.shape[1]
        h = n**(-1/(d+1))
        model = MixtureDensityNetwork('h-{}'.format(i), ndim_x, ndim_y, n_centers=10, hidden_sizes=(16, 16), hidden_nonlinearity=tf.nn.tanh,
               n_training_epochs=1000, x_noise_std=h, y_noise_std=h, adaptive_noise_fn=None, entropy_reg_coef=0.0,
               weight_decay=0.0, weight_normalization=True, data_normalization=True, dropout=0.0, l2_reg=0.0, l1_reg=0.0,
               random_seed=42)
        
        # Fit
        model.fit(X_train, y_train)
        # Predict
        y_pred = model.mean_(X_test)
        y_pred = y_pred.reshape((-1,1))
        # Define discrepancy
        std = model.std_(X_test)
        discrepancy = np.divide((y_test-y_pred), 1+std)
        # Get dataframe for test observations with behavioral data + discrepancy
        data = {'discrepancy':discrepancy[:,0]}
        discrepancy_df = pd.DataFrame(data)
        ID_df = pd.DataFrame({'EID':ID_test})
        discrepancy_merged = pd.concat([ID_df, discrepancy_df], axis=1)
        df = pd.merge(discrepancy_merged, behavioral, how='inner', on='EID')
        
        print(df.columns)
        
        # Get the correlations
        correlations = df[df.columns[1:]].corr()['discrepancy'][:]      
        
        for feat in correlations[correlations > corr_threshold].index:
            if df[feat].isna().sum() < 100:
                print(feat)
                model = ols('{} ~ discrepancy + Age'.format(feat), data = df).fit()
                anova_result = sm.stats.anova_lm(model, typ=2)
    
                if pvals.get(feat, -1) == -1:
                    pvals[feat] = [anova_result['PR(>F)']['discrepancy']]
                else:
                    pvals[feat].append(anova_result['PR(>F)']['discrepancy'])
                    
    return pvals
        
    #print('Mean t-test p-val (inattentive vs healthy): {} \n Mean t-test p-val (combined vs healthy): {} \n Mean t-test p-val (autism vs healthy): {}, \n Mean t-test p-val (impaired vs healthy): {}'.format(np.mean(t_adhd), np.mean(t_adhd_combined), np.mean(t_autism), np.mean(t_impaired)))

In [155]:
res = discrepancy_corr(5, corr_threshold = 0.2)

1000/1000 [100%] ██████████████████████████████ Elapsed: 56s | loss: 645.285
mean log-loss train: 1.4404
discrepancy
Age
CTOPP_EL_R
CTOPP_BW_R
FGC_Curl_Up
PCIAT_Total
WIAT_Num_Raw
WIAT_Spell_Raw
WIAT_Word_Raw
WIAT_LCRV_Raw
WIAT_MP_Raw
WISC_BD_Raw
WISC_Similarities_Raw
WISC_MR_Raw
WISC_Vocab_Raw
1000/1000 [100%] ██████████████████████████████ Elapsed: 59s | loss: 633.623
mean log-loss train: 1.4143
discrepancy
Age
CTOPP_EL_R
CTOPP_BW_R
FGC_Curl_Up
PCIAT_Total
WIAT_Num_Raw
WIAT_Spell_Raw
WIAT_LCRV_Raw
WIAT_MP_Raw
WISC_BD_Raw
WISC_Similarities_Raw
WISC_Vocab_Raw
WISC_VP_Raw
1000/1000 [100%] ██████████████████████████████ Elapsed: 60s | loss: 665.333
mean log-loss train: 1.4851
discrepancy
Age
CTOPP_EL_R
CTOPP_BW_R
FGC_Curl_Up
WIAT_Num_Raw
WIAT_Pseudo_Raw
WIAT_Spell_Raw
WIAT_Word_Raw
WIAT_LCRV_Raw
WIAT_MP_Raw
WISC_BD_Raw
WISC_Similarities_Raw
WISC_MR_Raw
WISC_Vocab_Raw
WISC_VP_Raw


Using the Benjamini-Hochberg correction.

In [165]:
model = ols('CTOPP_BW_R ~ discrepancy + Age', data = df).fit()

anova_result = sm.stats.anova_lm(model, typ=2)
anova_result

,sum_sq,df,F,PR(>F)
discrepancy,0.408809,1.0,0.016510,8.978141e-01
Age,2169.274895,1.0,87.608667,3.229591e-19
Residual,11711.935090,473.0,NaN,NaN


In [158]:
pvals

{'discrepancy': [0.0],
 'Age': [1.0],
 'CTOPP_EL_R': [0.7605612970528647],
 'CTOPP_BW_R': [0.8978141472047116],
 'FGC_Curl_Up': [0.4141129979267989],
 'PCIAT_Total': [0.3882321985743732],
 'WIAT_Num_Raw': [0.09744599509459974],
 'WIAT_Spell_Raw': [0.15299767574839895],
 'WIAT_Word_Raw': [0.09061227711565088],
 'WIAT_LCRV_Raw': [0.05194419477173275],
 'WIAT_MP_Raw': [0.06642134014895226],
 'WISC_BD_Raw': [0.861677651437721]}